Run: docker compose up -d

In [10]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointsSelector, PointIdsList

client = QdrantClient(host="qdrant_nosql_lab", port=6333)


### Tworzenie / nadpisanie kolekcji

In [11]:
client.recreate_collection(
    collection_name="my_collection",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)


/tmp/ipykernel_336/332858613.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

### Sprawdzanie czy kolekcja istnieje

In [12]:
exists = client.collection_exists("my_collection")
print("Istnieje:", exists)


Istnieje: True


### Lista wszystkich kolekcji

In [13]:
collections = client.get_collections()
print(collections)

collections=[CollectionDescription(name='my_collection')]


### Dodawanie punktów (danych)

In [14]:
from qdrant_client.models import PointStruct

# Przygotowanie punktów do dodania:
# Każdy punkt to obiekt PointStruct z:
# - unikalnym ID (np. 0, 1, 2...),
# - wektorem (embeddingiem, np. 384-wymiarowym),
# - opcjonalnym payloadem (np. tekst lub metadane)

points = [
    PointStruct(
        id=0,                                # unikalny identyfikator punktu
        vector=[0.1] * 384,                  # 384-wymiarowy wektor (np. embedding zdania)
        payload={"text": "przykład 1"}       # dodatkowe dane (można filtrować po payloadach)
    ),
    PointStruct(
        id=1,
        vector=[0.2] * 384,
        payload={"text": "przykład 2"}
    )
]

# Dodanie punktów do kolekcji o nazwie 'my_collection'
# Jeśli punkty o tych ID już istnieją – zostaną nadpisane (upsert = insert + update)
client.upsert(
    collection_name="my_collection",
    points=points
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Wyszukiwanie podobnych wektorów

In [15]:
query_vector = [0.1]*384 # Wektor o długości 384 i wartościach 0.1

results = client.search(
    collection_name="my_collection",
    query_vector=query_vector,
    limit=3,
    with_payload=True
)

for result in results:
    print(f"ID: {result.id}, score: {result.score}, payload: {result.payload}")


ID: 0, score: 0.9999999, payload: {'text': 'przykład 1'}
ID: 1, score: 0.9999999, payload: {'text': 'przykład 2'}


/tmp/ipykernel_336/1154867874.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


### Usuwanie punktów

In [16]:
client.delete(
    collection_name="my_collection",
    points_selector=PointIdsList(points=[0, 1])
)


UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

### Usuwanie kolekcji

In [18]:
client.delete_collection(collection_name="my_collection")

True